Script to average kelp measurements onto same grid as MUR data

In [1]:
import os
import glob
import numpy as np
import xarray as xr
from tqdm import tqdm
from scipy.interpolate import interp1d
from joblib import dump

r_earth = 6371.0 # km

kelp_file = "/home/jovyan/efs/data/KelpForest/LandsatKelpBiomass_2022_Q4_withmetadata.nc"
kelp = xr.open_dataset(kelp_file)

mur_dir = "/home/jovyan/efs/data/MUR"
mur_files = glob.glob(os.path.join(mur_dir, "*MUR*.nc"))

In [2]:
def process_location(lat, lon, kelp):
    # average kelp area within 0.01 degrees of the lat/lon

    location_data = {
        'lat': lat,
        'long': lon,
        'mur_time': [],
        'mur_temp': [],
        'mur_temp_std': [],
        'kelp_area': [], # m^2 per km^2
        'kelp_time': kelp.time.values,
    }

    mask = (kelp.latitude >= lat - 0.005) & (kelp.latitude < lat + 0.005) & \
           (kelp.longitude >= lon - 0.005) & (kelp.longitude < lon + 0.005)

    if not kelp.time.values.size:  # Check if 'kelp_time' is empty
        return None

    if not np.any(mask) or np.all(np.isnan(kelp.area.values[:, mask])):
        return None
    
    # calculate the area of the grid cell adjust for latitude
    scale_factor = r_earth * np.pi / 180.0  * np.abs(np.cos(np.deg2rad(lat))) # km per degree
    area = (scale_factor * 0.01 * 1000)**2 # meters squared
    location_data['kelp_area'] = np.nansum(kelp.area.values[:, mask],axis=1) # total surface area [m^2]
    location_data['kelp_area'] = location_data['kelp_area'] / area * 1000**2 # m^2 per km^2

    return location_data

In [3]:

print("Reading and processing data...")
data = []
ds = xr.open_dataset(mur_files[0])
lat_grid = ds.lat.values
lon_grid = ds.lon.values
ds.close()

# Find the minimum and maximum latitude and longitude values from the kelp dataset
min_lat, max_lat = kelp.latitude.min().values, kelp.latitude.max().values
min_lon, max_lon = kelp.longitude.min().values, kelp.longitude.max().values

# Filter the lat_grid and lon_grid arrays to include only the values within the kelp's latitude and longitude range
lat_mask = (lat_grid >= min_lat) & (lat_grid <= max_lat)
lon_mask = (lon_grid >= min_lon) & (lon_grid <= max_lon)
lat_grid = lat_grid[lat_mask]
lon_grid = lon_grid[lon_mask]

print("Extracting kelp data...")
for i in tqdm(enumerate(lat_grid), total=len(lat_grid)):
    lat = i[1]
    for j,lon in enumerate(lon_grid):
        location_data = process_location(lat, lon, kelp)
        if location_data is not None:
            data.append(location_data)

# loop over temperature files and add to data
print("Reading SST data...")
for file in tqdm(mur_files):
    ds = xr.open_dataset(file)
    for i, location in enumerate(data):
        lat, lon = location['lat'], location['long']
        location['mur_temp'].append(ds.sel(lat=lat, lon=lon).monthly_mean_sst.values)
        location['mur_temp_std'].append(ds.sel(lat=lat, lon=lon).monthly_std_sst.values)
        location['mur_time'].append(ds.time.values[0])
    ds.close()

# convert to numpy arrays
for location in data:
    location['mur_temp'] = np.array(location['mur_temp'])
    location['mur_temp_std'] = np.array(location['mur_temp_std'])
    location['mur_time'] = np.array(location['mur_time'])

# remove second axis on tmp and tmp_std
for location in data:
    location['mur_temp'] = np.squeeze(location['mur_temp'])
    location['mur_temp_std'] = np.squeeze(location['mur_temp_std'])

print("Interpolating SST data onto kelp time grid...")
data = [interpolate_data(d) for d in tqdm(data)]

with open(f'Data/kelp_averaged_data.pkl', 'wb') as f:
    dump(data, f)


Reading and processing data...
Extracting kelp data...


 81%|████████  | 1723/2139 [1:09:52<16:52,  2.43s/it]
Exception ignored in: <bound method IPythonKernel._clean_thread_parent_frames of <ipykernel.ipkernel.IPythonKernel object at 0x7fbe10238690>>
Traceback (most recent call last):
  File "/srv/conda/envs/notebook/lib/python3.11/site-packages/ipykernel/ipkernel.py", line 770, in _clean_thread_parent_frames
    def _clean_thread_parent_frames(

KeyboardInterrupt: 
Exception ignored in: <bound method IPythonKernel._clean_thread_parent_frames of <ipykernel.ipkernel.IPythonKernel object at 0x7fbe10238690>>
Traceback (most recent call last):
  File "/srv/conda/envs/notebook/lib/python3.11/site-packages/ipykernel/ipkernel.py", line 770, in _clean_thread_parent_frames
    def _clean_thread_parent_frames(

KeyboardInterrupt: 


KeyboardInterrupt: 

In [ ]:
len(data)

In [ ]:
data[0]

Run the script below to clean the data and convert it into usable metrics/features for regression

need to re-make this everytime data changes

In [ ]:
!python kelp_metrics.py